<a href="https://colab.research.google.com/github/sutiliza-org/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import requests
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from google.colab import drive
import warnings

warnings.filterwarnings('ignore')

# --- Configurações e Constantes ---
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "graficos")  # Pasta para gráficos
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive")  # Pasta raiz para o sumário
DATASET_URL = "https://docs.google.com/spreadsheets/d/13IWvMp6SA7GCJybJVcw92EN-NaQC4bRSWeP8Y_YPH1Y/export?format=csv"
DEFAULT_PALETTE = "viridis"
FIGSIZE = (10, 6)

# Estilo dos gráficos
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 18,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# --- Funções Auxiliares ---
def save_fig(fig, filename):
    """Salva a figura no Drive, na pasta 'graficos'."""
    filepath = os.path.join(GRAPHPATH, filename)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)  # Garante que a pasta existe
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Fecha a figura *imediatamente* após salvar

def create_figure():
    """Cria uma figura com o tamanho padrão."""
    return plt.figure(figsize=FIGSIZE)  # Retorna a figura diretamente

def load_data_from_url(url):
    """Carrega dados de uma URL CSV."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_content = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content))
        return df
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

# --- Função para Salvar Sumário ---
def generate_statistical_summary(df, filename="sumario.txt"):
    """Gera um sumário estatístico e salva em um arquivo na pasta raiz do Google Drive."""
    try:
        summary = df.describe().to_string()
        filepath = os.path.join(SUMMARYPATH, filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)  # Garante que a pasta existe
        with open(filepath, 'w') as f:
            f.write(summary)
        print(f"Sumário estatístico salvo em: {filepath}")
    except Exception as e:
        print(f"Erro ao gerar sumário estatístico: {e}")

# --- Funções de Visualização (Refatoradas - Simplificadas) ---
def generate_correlation_heatmap(df, title, filename):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)  # Adiciona um único Axes
    corr = df.corr()
    sns.heatmap(corr, annot=True, cmap=DEFAULT_PALETTE, fmt=".2f", linewidths=.5, ax=ax)
    ax.set_title(title)
    save_fig(fig, filename)

def generate_scatterplot(df, x_col, y_col, title, filename, hue=None, style=None):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)
    sns.scatterplot(x=x_col, y=y_col, data=df, hue=hue, style=style, palette=DEFAULT_PALETTE, alpha=0.7, ax=ax)
    ax.set_title(title)
    save_fig(fig, filename)

def generate_histogram(df, col, title, filename, bins=20, kde=True):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)
    sns.histplot(df[col], kde=kde, bins=bins, color='skyblue', ax=ax)
    ax.set_title(title)
    save_fig(fig, filename)

def generate_boxplot(df, x_col, y_col, title, filename, hue=None):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)
    if hue:
        sns.boxplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE, ax=ax)
    else:
        sns.boxplot(x=x_col, y=y_col, data=df, palette=DEFAULT_PALETTE, ax=ax)
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=45)
    save_fig(fig, filename)

def generate_violinplot(df, x_col, y_col, title, filename, hue=None, split=False):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)
    sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, split=split, palette=DEFAULT_PALETTE, ax=ax)
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=45)
    save_fig(fig, filename)

def generate_pairplot(df, title, filename, hue=None):
    pairplot = sns.pairplot(df, hue=hue, palette=DEFAULT_PALETTE)
    pairplot.fig.suptitle(title, y=1.02)
    save_fig(pairplot.fig, filename)  # Salva pairplot.fig

def generate_pca_plot(df, features, title, filename, hue=None, n_components=2):
    fig = create_figure()
    ax = fig.add_subplot(1, 1, 1)
    df_pca = df[features].dropna()
    if df_pca.empty:
        print("Sem dados para PCA.")
        return
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_pca)
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(scaled_data)
    pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(n_components)])
    if hue and hue in df.columns:
        pca_df = pca_df.set_index(df_pca.index)
        pca_df[hue] = df[hue].loc[pca_df.index]
    sns.scatterplot(x='PC1', y='PC2', data=pca_df, hue=hue, palette=DEFAULT_PALETTE, ax=ax)
    ax.set_title(title)
    save_fig(fig, filename)

# --- Função Principal (main) ---
if __name__ == "__main__":
    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
    print(f"Drive montado em: {DRIVE_MOUNT_PATH}")

    # Garante que as pastas existem
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)

    # Carrega os dados
    df = load_data_from_url(DATASET_URL)
    if df is None:
        print("Não foi possível carregar os dados. Encerrando.")
        exit()

    # --- Análises e Visualizações (12+ Gráficos) ---
    generate_correlation_heatmap(df, "1. Heatmap de Correlação (Completo)", "1_heatmap_completo.png")
    generate_scatterplot(df, 'Tempo Codificação (min)', 'Atividade Alfa (µV²)', "2. Tempo de Codificação vs. Atividade Alfa", "2_scatter_tempo_alfa.png")
    generate_scatterplot(df, 'Erros', 'Atividade Beta (µV²)', "3. Erros vs. Atividade Beta", "3_scatter_erros_beta.png")
    generate_histogram(df, 'Tempo Codificação (min)', "4. Distribuição do Tempo de Codificação", "4_hist_tempo.png")
    generate_histogram(df, 'Erros', "5. Distribuição de Erros", "5_hist_erros.png")

    cluster_features = ['Tempo Codificação (min)', 'Erros', 'Complexidade Ciclomática']
    clusters = perform_kmeans_clustering(df, cluster_features, n_clusters=3)
    if clusters is not None:
        df['Cluster'] = clusters
        generate_boxplot(df, 'Cluster', 'Tempo Codificação (min)', "6. Tempo de Codificação por Cluster", "6_boxplot_tempo_cluster.png")

    pca_features = ['Atenção Sustentada (Escore)', 'Memória de Trabalho (Escore)', 'Controle Inibitório (Escore)']
    generate_pca_plot(df, pca_features, "7. PCA Plot", "7_pca_plot.png")
    generate_scatterplot(df, 'Complexidade Ciclomática', 'Tempo Codificação (min)', "8. Complexidade vs. Tempo (por Cluster)", "8_scatter_complexidade_tempo.png", hue='Cluster')
    generate_violinplot(df, 'Cluster', 'Atividade Alfa (µV²)', "9. Atividade Alfa por Cluster", "9_violin_alfa_cluster.png")
    generate_boxplot(df, 'Cluster', 'Erros', "10. Erros por Cluster", "10_boxplot_erros_cluster.png")
    generate_scatterplot(df, 'Atenção Sustentada (Escore)', 'Memória de Trabalho (Escore)', "11. Atenção Sustentada vs. Memória de Trabalho", "11_scatter_atencao_memoria.png")
    generate_histogram(df, 'Complexidade Ciclomática', "12. Distribuição da Complexidade Ciclomática", "12_hist_complexidade.png")

    pairplot_vars = ['Tempo Codificação (min)', 'Erros', 'Atividade Alfa (µV²)', 'Atividade Beta (µV²)']
    generate_pairplot(df[pairplot_vars], "13. Pairplot (Variáveis Selecionadas)", "13_pairplot.png")

    # --- Análise Estatística e Sumário ---
    calculate_correlations(df, 'Tempo Codificação (min)', 'Atividade Alfa (µV²)')
    calculate_correlations(df, 'Erros', 'Atividade Beta (µV²)')

    regression_features = [
        'Atividade Alfa (µV²)', 'Atividade Beta (µV²)', 'Atenção Sustentada (Escore)',
        'Memória de Trabalho (Escore)', 'Controle Inibitório (Escore)', 'Complexidade Ciclomática',
        'Erros'
    ]
    perform_linear_regression(df, regression_features, 'Tempo Codificação (min)')

    generate_statistical_summary(df, "sumario.txt")  # Gera o sumário na pasta raiz

    drive.flush_and_unmount()
    print("Análise completa, gráficos e sumário gerados!")

Mounted at /content/drive
Drive montado em: /content/drive
Correlação de Pearson (Tempo Codificação (min) vs. Atividade Alfa (µV²)): 0.993, p=0.000
Correlação de Spearman (Tempo Codificação (min) vs. Atividade Alfa (µV²)): 0.998, p=0.000
Correlação de Pearson (Erros vs. Atividade Beta (µV²)): -0.941, p=0.000
Correlação de Spearman (Erros vs. Atividade Beta (µV²)): -0.982, p=0.000
                               OLS Regression Results                              
Dep. Variable:     Tempo Codificação (min)   R-squared:                       0.996
Model:                                 OLS   Adj. R-squared:                  0.995
Method:                      Least Squares   F-statistic:                     1155.
Date:                     Sun, 09 Feb 2025   Prob (F-statistic):           1.35e-36
Time:                             07:06:04   Log-Likelihood:                -51.572
No. Observations:                       40   AIC:                             119.1
Df Residuals:                